In [1]:
# https://huggingface.co/blog/how-to-train

In [2]:
!pip install datasets
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [1]:
import transformers
from transformers import RobertaConfig
from transformers import RobertaForMaskedLM
from transformers import Trainer, TrainingArguments
from tokenizers import ByteLevelBPETokenizer
from transformers import DataCollatorForLanguageModeling
from transformers import RobertaTokenizerFast
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import SequentialSampler
from datasets import load_dataset
import os
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/arianabritez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#ds = load_dataset("marianbasti/boletin-oficial-argentina")
ds = load_dataset("piuba-bigdata/articles_and_comments")

Found cached dataset parquet (/Users/arianabritez/.cache/huggingface/datasets/piuba-bigdata___parquet/piuba-bigdata--articles_and_comments-94c404d9208411a3/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['tweet_id', 'text', 'title', 'url', 'user', 'body', 'created_at', 'comments'],
        num_rows: 537201
    })
})

In [4]:
ds["train"][0]

{'tweet_id': '1376940813968609288',
 'text': 'Segunda ola de coronavirus: preocupan las reuniones sociales y el Gobierno analiza medidas para después de Semana Santa https://t.co/KG2XU0uRKz',
 'title': 'Segunda ola de coronavirus: preocupan las reuniones sociales y el Gobierno analiza medidas para después de Semana Santa',
 'url': 'https://www.clarin.com/politica/segunda-ola-coronavirus-preocupan-reuniones-sociales-gobierno-analiza-medidas-despues-semana-santa_0_NnjHl1iUe.html?utm_medium=Social&utm_source=Twitter#Echobox=1617123246',
 'user': 'clarincom',
 'body': 'La decisión de Axel Kicillof de suspender actividades nocturnas y limitar las reuniones sociales no sorprendió a la Casa Rosada. Más allá de que había avisado que tomaría medidas restrictivas, el gobernador bonaerense se adelantó a la primera herramienta a la que apelaría la administración de Alberto Fernández después de Semana Santa si es que la curva de contagios sigue en ascenso.\n\nLa confirmación de que en Argentina ya 

In [5]:
#the news article is in the body key
ds["train"][4]["body"]

'La noche del domingo 28 de marzo efectivos que responden a la fuerza comandada por el ministro Sergio Berni esposaron a una trabajadora de la salud que se aprestaba a socorrer a una persona que llegó con dolor en el pecho a la UPA N° 5 de Longchamps, en el distrito de Almirante Brown, al sur del conurbano bonaerense.\n\nEl motivo esgrimido por los uniformados fue que ella, al cumplir con su tarea, no estaba haciendo lo que ellos le ordenaron, que era extraer sangre a una persona que llevaron al lugar. Con total prepotencia esgrimieron que estaba "resistiendo a la autoridad". Cabe aclarar que en ese lugar no se realizan extracciones, según explicó la misma enfermera, que luego del incidente -durante el cual sus compañeros reclamaban que fuera dejada en libertad- debió ser atendida presa del malestar nervioso que le provocó toda la situación.\n\nEn una entrevista con Canal 13 este martes la profesional explicó que "Fue de terror lo que me pasó, somos pocas enfermeras en las guardias por

In [28]:
feature_to_keep = "body"

#remove all columns except the body of text
updated_ds = ds.map(\
    lambda x: x,  # the data stays the same
    remove_columns=[col for col in ds['train'].column_names if col != feature_to_keep]
)

Loading cached processed dataset at /Users/arianabritez/.cache/huggingface/datasets/piuba-bigdata___parquet/piuba-bigdata--articles_and_comments-94c404d9208411a3/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-d182953cb8ddcf51.arrow


In [29]:
updated_ds

DatasetDict({
    train: Dataset({
        features: ['body'],
        num_rows: 537201
    })
})

In [30]:
updated_ds["train"]

Dataset({
    features: ['body'],
    num_rows: 537201
})

In [31]:
df = pd.DataFrame(updated_ds["train"])

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390292 entries, 0 to 390291
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   title            390292 non-null  object        
 1   name             390292 non-null  object        
 2   entity           390292 non-null  object        
 3   summary          390292 non-null  object        
 4   full_text        390292 non-null  object        
 5   url_in_articles  390292 non-null  object        
 6   date             390292 non-null  datetime64[ns]
 7   url              390292 non-null  object        
dtypes: datetime64[ns](1), object(7)
memory usage: 23.8+ MB


In [12]:
df.isnull().sum()

,0
title,0
name,0
entity,0
summary,0
full_text,0
url_in_articles,0
date,0
url,0


In [13]:
df = df[:20000]

In [14]:
# Tokenizing the text column with list comprehension
df_tokens = df
df_tokens['tokens'] = [nltk.word_tokenize(text) for text in df_tokens['full_text']]

# Count total number of tokens
total_tokens = sum(len(tokens) for tokens in df_tokens['tokens'])

print(f"Total number of tokens: {total_tokens}")

Total number of tokens: 7799539


In [15]:
sequences = df['full_text'].tolist()

In [16]:
##Celonie's code starts here

#train_filepath = 'data/ranked_sequences.csv'


#ranked = pd.read_csv(train_filepath)
#sequences = ranked['sequence'].tolist()

#with open('combined.dev') as infile:
#  devset = infile.read().split("\n")[:10000]


In [17]:
len(sequences)

20000

In [18]:
#split data into train and dev

df_train, df_dev = train_test_split(sequences, test_size=0.15, random_state=None)

In [19]:
len(df_train)

17000

In [20]:
tokenizer_folder = 'tokenizer_folder'

if not os.path.exists(tokenizer_folder):
    os.mkdir(tokenizer_folder)

In [21]:

# Initialize tokenizer
tokenizer = ByteLevelBPETokenizer()

tokenizer.train_from_iterator(df_train, vocab_size=52_000, min_frequency=2, show_progress=True, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

# Save tokenizer
tokenizer.save_model(tokenizer_folder)

['tokenizer_folder/vocab.json', 'tokenizer_folder/merges.txt']

In [22]:

# Configuration for RoBERTa model
config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)
# Initialize model
model = RobertaForMaskedLM(config=config)
print('Num parameters: ',model.num_parameters())

Num parameters:  83504416


In [23]:
# tokenizer_folder = 'tokenizer_folder'

# Load tokenizer
max_length = 512
tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_folder, max_len=max_length)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [24]:
# create CustomDataset class

class CustomDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.examples = []
        self.mask = []

        for example in data:
            x=tokenizer.encode_plus(example, max_length = max_length, truncation=True, padding=True)
            self.examples += [x.input_ids]
            self.mask += [x.attention_mask]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return torch.tensor(self.examples[i])

# create train and evaluation datasets
train_dataset = CustomDataset(df_train, tokenizer)
eval_dataset = CustomDataset(df_dev, tokenizer)

In [25]:

# Define data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [26]:
#adapted from https://github.com/MiuLab/FastMTL/blob/only3/custom_trainer.py

# adapt get_train_dataloader function to supply DataLoader using SequentialSampler and shuffle=False to enforce curriculum learning

from transformers.trainer import *
def get_train_dataloader(self) -> DataLoader:
    """
    Returns the training :class:`~torch.utils.data.DataLoader`.

    Will use no sampler if :obj:`self.train_dataset` does not implement :obj:`__len__`, a random sampler (adapted
    to distributed training if necessary) otherwise.

    Subclass and override this method if you want to inject some custom behavior.
    """
    if self.train_dataset is None:
        raise ValueError("Trainer: training requires a train_dataset.")

    return DataLoader(
        self.train_dataset,
        batch_size=self.args.train_batch_size,
        sampler=SequentialSampler(self.train_dataset),
        collate_fn=self.data_collator,
        drop_last=self.args.dataloader_drop_last,
        num_workers=self.args.dataloader_num_workers,
        shuffle=False
    )
Trainer.get_train_dataloader = get_train_dataloader

In [ ]:
batch_size = 16

# Define training arguments
training_args = TrainingArguments(
    output_dir='model_folder',
    overwrite_output_dir=True,
    evaluation_strategy = 'epoch',
    num_train_epochs=5,
    learning_rate=1e-4,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_steps=8192,
    save_total_limit=1,
    #seed=10098,
    max_steps=int(1269227 / batch_size)
)
# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)
# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,5.290600,5.080783
2,4.896800,4.808399
3,4.678500,4.598981
4,4.452500,4.365621
5,4.234000,4.006622
6,3.779100,3.449561
7,3.312100,3.079302
8,2.849500,2.840581
